In [6]:
print('Validate nvidia-smi (software managemtn interface):')
!nvidia-smi
print()
print('Validate nvcc --version:')
!nvcc --version

Validate nvidia-smi (software managemtn interface):
Mon Oct 16 06:52:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   34C    P0    12W /  N/A |      5MiB /  3910MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                    

In [12]:
# Below is from previous project 2: Dataset
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

DATADIR = "/home/peter/SVU/SVU_CS512_DL/00_kaggle_cat_dog/PetImages"
CATEGORIES = ["Dog", "Cat"] 
IMG_SIZE = 50
training_data = []
def create_training_data():
    for category in CATEGORIES:  # do dogs and cats
        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat
        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))
create_training_data()
print(len(training_data))

import random
random.shuffle(training_data)
X = []
y = []
for features,label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # -1: keep all rows with image size, 1: grayscale
print('X.shape:', X.shape)
y = np.array(y) # convert array into np.array (tensor) for tensorflow 2.0
import pickle
pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

100%|███████████████████████████████████| 12501/12501 [00:08<00:00, 1482.32it/s]


24946
X.shape: (24946, 50, 50, 1)


In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import pickle
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)
pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)
X = X/255.0      # X is already in tensor for tensorflow 2.0
#y = np.array(y)  # convert array into np.array (tensor) for tensorflow 2.0
model = Sequential()
print('X.shape:', X.shape)
# X.shape = (24946, 50, 50, 1)
print('input_shape: X.shape[1:]:', X.shape[1:])
# NN (neural Network) layer 1: 
# ConvNets layers 1-3 (Conv2D, Actionvation, MaxPooling2D)
# input_shape: X.shape[1:]: (50, 50, 1)
model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# NN Layer 2: ConvNets layers 4-6 (Conv2D, Actionvation, MaxPooling2D)
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# NN Layer 3: ConvNets layers 7-8 (Flatten, Dense) 
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
# Layer 4: ConvNets layer 9-10 (Dense, Activation)
model.add(Dense(1))
model.add(Activation('sigmoid'))
# Set up parameter
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print('X.shape:', X.shape)
print('y.shape:', y.shape)
# X.shape: (24946, 50, 50, 1)
# y.shape: (24946,)

X.shape: (24946, 50, 50, 1)
input_shape: X.shape[1:]: (50, 50, 1)
X.shape: (24946, 50, 50, 1)
y.shape: (24946,)


In [11]:
print('X.shape:', X.shape)
print('y.shape:', y.shape)
# X.shape: (24946, 50, 50, 1)
# y.shape: (24946,)
model.fit(X, y, batch_size=10, validation_split=0.3)

X.shape: (24946, 50, 50, 1)
y.shape: (24946,)


2023-10-16 09:52:12.740562: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 174620000 exceeds 10% of free system memory.


1747/1747 [==============================] - 272s 155ms/step - loss: 0.6680 - accuracy: 0.5921 - val_loss: 0.6478 - val_accuracy: 0.6387


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

import pickle

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0
y = np.array(y)

model = Sequential()
# X.shape = (24946, 50, 50, 1)
model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=30, epochs = 3, validation_split=0.3)

Epoch 1/3
583/583 [==============================] - 291s 499ms/step - loss: 0.6265 - accuracy: 0.6463 - val_loss: 0.6805 - val_accuracy: 0.6025
Epoch 2/3
583/583 [==============================] - 247s 424ms/step - loss: 0.5438 - accuracy: 0.7286 - val_loss: 0.5260 - val_accuracy: 0.7374
Epoch 3/3
583/583 [==============================] - 242s 414ms/step - loss: 0.5079 - accuracy: 0.7536 - val_loss: 0.5174 - val_accuracy: 0.7449
